# 客户发展关系的马尔可夫过程模型及其应用

在这个项目中，我们研究了十一年的购买记录，根据RFM模型中的近度指标对客户进行了细分，并计算出一个转移概率矩阵矩阵来对其行为进行建模。

In [1156]:
# import modules
import itertools as it
from collections import Counter

import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'
# load data and set column labels
df = pd.read_table('purchases.txt', header=None)
df.columns = ['user_id', 'amount', 'date']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/cheriehe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


In [1157]:
# examine data
df.head(3)

,user_id,amount,date
0,760,25.0,2009-11-06
1,860,50.0,2012-09-28
2,1200,100.0,2005-10-25


In [1158]:
# 51243条数据
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51243 entries, 0 to 51242
Data columns (total 3 columns):
user_id    51243 non-null int64
amount     51243 non-null float64
date       51243 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 1.2+ MB


In [1159]:
## Deal with datetime issues

# convert date to datetime
df['date'] = pd.to_datetime(df.date)

# set date as the index
df = df.set_index('date')

# convert dates to years
df = df.to_period('A')

# reset index
df = df.reset_index()

#### 存在记录的客户共有18417个


In [1160]:
len(df.groupby(df['user_id']).count())

18417

In [1629]:
df.groupby(df['user_id']).count().sample(3)

,date,amount
user_id,,
92590,1,1
216660,1,1
165540,2,2


In [1162]:
df.head()

,date,user_id,amount
0,2009,760,25.0
1,2012,860,50.0
2,2005,1200,100.0
3,2009,1420,50.0
4,2013,1940,70.0


In [1163]:
# examine descriptive statistics
df.describe()

,user_id,amount
count,51243.000000,51243.000000
mean,108934.547938,62.337195
std,67650.610139,156.606801
min,10.000000,5.000000
25%,57720.000000,25.000000
50%,102440.000000,30.000000
75%,160525.000000,60.000000
max,264200.000000,4500.000000


In [1164]:
# get purchase frequency matrix
freq = pd.crosstab(df.user_id, df.date)

# examine purchase frequency matrix
freq.head()

date,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
user_id,,,,,,,,,,,
10,1,0,0,0,0,0,0,0,0,0,0
80,1,0,1,0,1,0,1,0,1,1,1
90,1,1,1,1,1,1,1,2,1,0,0
120,0,0,0,0,0,0,0,1,0,0,0
130,1,0,1,0,0,0,0,0,0,0,0


In [1165]:
# encode frequency segments
for y in freq.columns:
    freq.loc[:, y] = freq.loc[:, y].apply(lambda x: 2 if x > 1 else x)

In [1166]:
freq.head()

date,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
user_id,,,,,,,,,,,
10,1,0,0,0,0,0,0,0,0,0,0
80,1,0,1,0,1,0,1,0,1,1,1
90,1,1,1,1,1,1,1,2,1,0,0
120,0,0,0,0,0,0,0,1,0,0,0
130,1,0,1,0,0,0,0,0,0,0,0


对于最近一次消费的近度R，我们确定了6种状态：
-	0: 从未买过公司的产品
-	1: 当前时间段买了公司的产品
-	2: （当前时段没有买）在过去的一个时间段内买了公司的产品
-	3: （当前时段和过去的一个时间段内没有买）过去两个时间段内买了公司的产品
-	4: （当前时段过去的两个时间段内没有买）过去三个时间段内买了公司的产品
-	5: （当前时段与过去的一个、两个或三个时间段内没有买）在过去的四个时间段之前（包括第四个）买了公司的产品(n>=4)
- 状态空间R:{0,1,2,3,4,5}

In [1523]:
R = freq.values.copy()
R[:,2]

array([0, 1, 1, ..., 0, 0, 0])

In [1599]:
# get recency states
R = freq.values.copy()
for i in range(n):
    for j in range(m):
        if (j == 0) and (freq.values[i, j] > 0):
            R[i, j] = 1
        elif j > 0:
            if freq.values[i, j] > 0:
                R[i, j] = 1
            elif (freq.values[i, j-1] > 0):
                R[i, j] = 2
            elif((j-1>0)&(freq.values[i, j-2] > 0)):
                R[i,j]=3
            elif((j-2>0)&(freq.values[i, j-3] > 0)):
                R[i, j] = 4              
            elif((j-3>0)&(freq.values[i, j-4] > 0)):
                R[i, j] = 5
            elif((j-4>0)&(freq.values[i, j-5] > 0)):
                R[i, j] = 5
            elif((j-5>0)&(freq.values[i, j-6] > 0)):
                R[i, j] = 5
            elif((j-6>0)&(freq.values[i, j-7] > 0)):
                R[i, j] = 5
            elif((j-7>0)&(freq.values[i, j-8] > 0)):
                R[i, j] = 5
            elif((j-8>0)&(freq.values[i, j-9] > 0)):
                R[i, j] = 5
            elif((j-9>0)&(freq.values[i, j-10] > 0)):
                R[i, j] = 5
            elif((j-10>0)&(freq.values[i, j-11] > 0)):
                R[i, j] = 5
            

np.unique(R)

array([0, 1, 2, 3, 4, 5])

In [1638]:
# examine R
recency=pd.DataFrame(R)
# recency.columns = ['x0','x1','x2','x3','x4','x5','x6','x7','x8','x9','x10']
recency.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,2,3,4,5,5,5,5,5,5,5
1,1,2,1,2,1,2,1,2,1,1,1
2,1,1,1,1,1,1,1,1,1,2,3
3,0,0,0,0,0,0,0,1,2,3,4
4,1,2,1,2,3,4,5,5,5,5,5


In [1813]:
recency.groupby(recency[0]).count()

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
0,17192,17192,17192,17192,17192,17192,17192,17192,17192,17192
1,1225,1225,1225,1225,1225,1225,1225,1225,1225,1225


In [1814]:
recency.groupby(recency[0]).count()[1]/sum(recency.groupby(recency[0]).count()[1])

0
0    0.933485
1    0.066515
Name: 1, dtype: float64

#### 得到初始概率分布
$P_1 = 0.0665, P_0=0.9335, P_i=0(i\geq2)$

In [1672]:
1225/(17192+1225)

0.06651463321930824

In [1666]:
1- 1225/(17192+1225)

0.9334853667806917

In [1671]:
0.0665+0.9335

1.0

In [1604]:
# recency.index=range(len(recency))
# recency.head()

In [1605]:
len(recency)

18417

In [1606]:
# initialize hashmap
state_map = dict()

# get possible states for m, f, r
# ms = [0, 1, 2]
# fs = [0, 1, 2]
rs = list(range(4))

# get cartesian product of m, f, r states
# states = it.product(rs)

# assign inactive states to 0
for state in states:
    if state[0] == 0:
        state_map[state] = 0

# assign states 1-21 to customers who have made purchases but not yet churned
state = 0
for r in range(0, 6):
    state_tuple = (r)
    state_map[state_tuple] = int(state/4)
    state += 1
    state_tuple = (r)
    state_map[state_tuple] = int(state/4)
    state += 1
    state_tuple = (r)
    state_map[state_tuple] = int(state/4)
    state += 1
    state_tuple = (r)
    state_map[state_tuple] = int(state/4)
    state += 1

# assign state 21 to the absorbing state (churn, recency greater than five)
for state in state_map.keys():
#     print(state)
    if state > 5:
        state_map[state] = 6

In [1608]:
state_map

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}

In [1609]:
# examine state_map values
unique_states = np.unique(list(state_map.values()))
unique_states

array([0, 1, 2, 3, 4, 5])

In [1610]:
# R into a single array
MFR = np.zeros((n, m, 1))
MFR[:, :, 0] = R.copy()

In [1611]:
R

array([[1, 2, 3, ..., 5, 5, 5],
       [1, 2, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 2, 3],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [1612]:
len(MFR)

18417

In [1614]:
# initialize state matrix
S = np.zeros((n, m), dtype=int)
for i in range(n):
    for j in range(m):
        # each entry of S is the corresponding tuple from MFR, passed through the state_map
        S[i, j] = state_map[MFR[i, j, :].astype(int)[0]]
#         int(state_map[tuple(MFR[i, j, :].astype(int))])

In [1808]:
# examine S
pd.DataFrame(S).head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,2,3,4,5,5,5,5,5,5,5
1,1,2,1,2,1,2,1,2,1,1,1
2,1,1,1,1,1,1,1,1,1,2,3
3,0,0,0,0,0,0,0,1,2,3,4
4,1,2,1,2,3,4,5,5,5,5,5


From this state matrix, transition matrices can be calculated by counting state changes between periods.

In [1616]:
# get num states
n_states = len(unique_states)
n_states 

6

In [1617]:
# initialize transition array. Each slice is a 22x22 transition matrix, one for each transition between periods
T_freq = np.zeros((n_states, n_states, 10), dtype=int)

for p in range(10):
    for r in range(n):
        i = S[r, p]
        j = S[r, p+1]
        T_freq[i, j, p] += 1

# examine one slice
T_freq[1:10, 1:10, 0]

array([[674, 551,   0,   0,   0],
       [  0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0]])

In [1618]:
T_freq[1][2]

array([ 551,  851, 2431, 1821, 2106, 1955, 1712, 2509, 2317, 1958])

Per period transition frequency matrices are collapsed into a single transition frequency matrix for the whole dataset. From this, a transition probability matrix is calculated.

In [1619]:
# sum over all periods
T_freq_total = T_freq.sum(axis=2)

# examine totals
T_freq_total

array([[73515, 17192,     0,     0,     0,     0],
       [    0, 21357, 18211,     0,     0,     0],
       [    0,  2949,     0, 13304,     0,     0],
       [    0,  1060,     0,     0, 10341,     0],
       [    0,   463,     0,     0,     0,  7947],
       [    0,   720,     0,     0,     0, 17111]])

In [1620]:
T_freq_total

array([[73515, 17192,     0,     0,     0,     0],
       [    0, 21357, 18211,     0,     0,     0],
       [    0,  2949,     0, 13304,     0,     0],
       [    0,  1060,     0,     0, 10341,     0],
       [    0,   463,     0,     0,     0,  7947],
       [    0,   720,     0,     0,     0, 17111]])

In [1817]:
pd.DataFrame(T_freq_total)

,0,1,2,3,4,5
0,73515,17192,0,0,0,0
1,0,21357,18211,0,0,0
2,0,2949,0,13304,0,0
3,0,1060,0,0,10341,0
4,0,463,0,0,0,7947
5,0,720,0,0,0,17111


In [1621]:
# get number of churned customers who returned
churn_returns = T_freq_total[-1, 1:][:-1].sum()
print('churn returns: {}'.format(churn_returns))

# get total transitions
total_transitions = T_freq_total[1:, 1:].sum()
print('total transitions: {}'.format(total_transitions))

# percent of purchases missed, assuming churners wouldn't return
# if they weren't marketed to
percent_missed_purchases = churn_returns / total_transitions
print('percent missed purchases: {}'.format(percent_missed_purchases))

churn returns: 720
total transitions: 93463
percent missed purchases: 0.007703583236146924


In [1622]:
# initialze transition probability matrix
T_prob = np.zeros(T_freq_total.shape)
# populate values
for r in range(n_states):
    T_prob[r, :] = T_freq_total[r, :]/T_freq[r, :].sum()
    
# convert the last row into an absorbing state
T_prob[-1, :] = 0
T_prob[-1, -1] = 1

# examine transition matrix    
T_prob

array([[0.81046667, 0.18953333, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.53975435, 0.46024565, 0.        , 0.        ,
        0.        ],
       [0.        , 0.18144343, 0.        , 0.81855657, 0.        ,
        0.        ],
       [0.        , 0.0929743 , 0.        , 0.        , 0.9070257 ,
        0.        ],
       [0.        , 0.05505351, 0.        , 0.        , 0.        ,
        0.94494649],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        ]])

In [1623]:
T_prob.shape

(6, 6)

## 转移概率矩阵P

In [1820]:
TP=pd.DataFrame(T_prob.round(2))
TP

,0,1,2,3,4,5
0,0.81,0.19,0.00,0.00,0.00,0.00
1,0.00,0.54,0.46,0.00,0.00,0.00
2,0.00,0.18,0.00,0.82,0.00,0.00
3,0.00,0.09,0.00,0.00,0.91,0.00
4,0.00,0.06,0.00,0.00,0.00,0.94
5,0.00,0.00,0.00,0.00,0.00,1.00


In [1802]:
def mutl(matr,n):
    '''计算同一矩阵的n次相乘'''
    P=matr
    for i in range(n-1):
        P=P@matr
    return P

In [1806]:
Pp3=mutl(TP,4).round(4)
Pp3

,0,1,2,3,4,5
0,0.4305,0.2796,0.1283,0.0966,0.0649,0.0000
1,0.0000,0.2214,0.1299,0.1412,0.1845,0.3229
2,0.0000,0.1017,0.0689,0.0655,0.0618,0.7020
3,0.0000,0.0450,0.0284,0.0377,0.0318,0.8571
4,0.0000,0.0155,0.0095,0.0112,0.0188,0.9450
5,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000


In [1804]:
P4=(TP@TP@TP@TP).round(4)
P4

,0,1,2,3,4,5
0,0.4305,0.2796,0.1283,0.0966,0.0649,0.0000
1,0.0000,0.2214,0.1299,0.1412,0.1845,0.3229
2,0.0000,0.1017,0.0689,0.0655,0.0618,0.7020
3,0.0000,0.0450,0.0284,0.0377,0.0318,0.8571
4,0.0000,0.0155,0.0095,0.0112,0.0188,0.9450
5,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000


In [1832]:
P100=mutl(TP,100).round(4)
P100

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,1.0


### 进行预测

预测的2009年的客户状态

In [1834]:
p0=np.array([0.9334853667806917,0.06651463321930824,0,0,0,0])
pred=pd.DataFrame(p0@mutl(TP,4).round(4).values)
pred

,0
0,0.401865
1,0.275462
2,0.128360
3,0.099753
4,0.073195
5,0.021464


2009年的客户状态真实值

In [1828]:
real = recency.groupby(recency[4]).count()[0]/sum(recency.groupby(recency[4]).count()[0])
real

4
0    0.514145
1    0.240159
2    0.098876
3    0.097790
4    0.032579
5    0.016452
Name: 0, dtype: float64

#### 误差分析

In [1830]:
from sklearn.metrics import mean_squared_error
import math
rmse = math.sqrt(mean_squared_error(pred.values, real))
print("The root mean squared error is {}.".format(rmse))

The root mean squared error is 0.052282729956662964.


100年以后客户的状态

In [1842]:
p0=np.array([0.9334853667806917,0.06651463321930824,0,0,0,0])
pred100=pd.DataFrame(p0@mutl(TP,100).round(4).values)
pred100.columns=[['after 100 years']]
pred100

,after 100 years
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,1.0
